## Mini-Projet: Séparation de sources hyperspectrale

Une image hyperspectrale est une image où un spectre contenant $D$ valeurs ou canaux est enregistré à chaque pixel. Cette modalité d'imagerie est très utilisée en imagerie satellitaire, où elle permet d'améliorer considérablement la résolution de la prise d'image. Dans le cas des images satellitaires, les pixels de l'image ont ainsi une résolution spatiale de l'ordre de la dizaine de mètres, de sorte que ces pixels mélangent les contributions des différents éléments présents sur la scène imagée (végétation, eau, minéraux du sol, habitations, routes, etc.). Nous nous intéressons ici à un problème, connu sous le nom de séparation de sources hyperspectrale, qui vise à identifier, pour une observation donnée, les spectres des constituants élémentaires présents dans une scène, ainsi que la proportion ("l'abondance") de ces constituants en chaque pixel de l'image.

L'objectif de ce mini-projet est d'implémenter un algorithme de séparation de source hyperspectral et de l'appliquer à une image satellitaire issue de la base de donnée SAMSON. L'image en question est de taille 95 x 95 pixels. Chaque pixel correspond à un spectre de 156 canaux, qui couvrent un intervalle de longueurs d'onde qui s'étend de 401 à 889 nm. La résolution spectrale est de 3.13 nm. Chaque spectre Y(x, y) mesuré au niveau du pixel (x, y) de l'image correspond à la combinaison linéaire des spectres *élémentaires* associés à l'eau, à la végétation et aux minéraux présents dans le sol:

$$
Y[x, y] = \alpha_{1} [x, y] m_1 + \alpha_2 [x, y] m_2 + (1 - \alpha_1[x, y] - \alpha_2[x, y]) m_3,
$$

où $\alpha_1[x, y]$ et $\alpha_2[x, y]$ correspondent aux proportions relatives d'eau et de végétation en surface à la position $[x, y]$, et $m_1$, $m_2$ et $m_3$ correspondent aux spectres élémentaires de l'eau, de la végétation et du sol minéral respectivement.



In [1]:
# -*- Encoding: Latin-1 -*-
#!/usr/bin/python

from math import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from math import factorial
from scipy.linalg import eigh, det, norm

In [2]:
data_dict = loadmat("Samson.mat")
a = data_dict['A'].reshape((95*95, 3))
end = data_dict['M']
data = np.dot(a, end.T)

## 1. Visualisation des données

Le script ci-dessous permet de charger les données de l'image. Implémenter une fonction python permettant de visualiser:
- le spectre mesuré en un pixel de l'image
- l'image obtenue à une longueur d'onde donnée

In [3]:
def display_spectrum(img, loc):
    
    """
    Display the spectrum at location (x, y)
    
    :param img: Hyperspectral image
    :type img: 3D numpy array
    :param loc: Pixel location
    :type loc: tuple
    
    """
    liste_lambda = np.arange(401, 889, 3.13)
    liste_intensite = img[loc[0], loc[1]]
    plt.figure("Local spectrum")
    plt.title("Local spectrum")
    plt.ylabel("Radiance")
    plt.plot(liste_lambda, liste_intensite)
    plt.show()

In [4]:
display_spectrum(data.reshape((95, 95, 156)), (50, 50))

In [5]:
def display_band(img, spectr_idx):
    
    """
    Display the image corresponding to the specified spectral band
    
    :param img: Hyperspectral image
    :type img: 3D numpy array
    :param spectr_idx: Indexof the spectral band to display
    :type loc: int
    
    """
    img_filtre = img[:,:,spectr_idx]
    plt.figure("Special band image")
    plt.title("Special band image")
    plt.gca().pcolormesh(img_filtre)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

In [6]:
display_band(data.reshape((95, 95, 156)), 10)

## 2. Réduction de dimension

Les spectres observés dans les données sont des vecteurs de dimension $D = 156$ et peuvent donc être représentés dans l'espace vectoriel $\mathbb{R}^D$. Ces spectres correspondent cependant à la combinaison linéaire de $K = 3$ spectres élémentaires, pondérés par les quantités positives $\alpha _1$, $\alpha_2$ et $\alpha_3$, qui correspondent à la proportion relative de chacun des spectres élémentaires. Par ailleurs, on a la relation de liaison supplémentaire:

$$
\alpha_1 + \alpha_2 + \alpha_3 = 1.
$$

Compte tenu de ces différentes relations, est-il possible de réduire la dimension des données, c'est à dire de déterminer un sous-espace de $\mathbb{R}^D$ qui contienne l'ensemble des points de données ? Quelle serait la dimension de cet espace ? En utilisant l'algorithme d'analyse en composantes principales (ACP, ou PCA en anglais) présenté dans les slides accompagnant le mini-projet, identifier le sous-espace en question. Projeter les données spectrales dans ce sous-espace, et afficher le résultat. Commenter le résultat. En particulier, comment explique-t-on la distribution géométrique des données spectrales?


In [7]:
def pca(data, K):

    """
    Principal component analysis (PCA)
    """
    # Compute the covariance matrix and the mean of the data
    cov = np.cov(data.T)
    mean = np.mean(data)

    # Computes the eigenvalues/eigenvectors of the covariance matrix
    eival, eivec = eigh(cov)

    # Extract the K largest eigenvectors
    eival_max = eival[-K:]
    eivec_max = eivec[-K:]
    # Project the data on the subpasce spanned by the K largest eigenvectors
    Y = data - mean
    Y = Y @ eivec_max.T
    #Y = ...
    #return Y
    return Y

In [8]:
projected_data = pca(data, 2)
print(projected_data)

[[ 0.313665   -0.08553223]
 [ 0.313665   -0.08553223]
 [ 0.313665   -0.08553223]
 ...
 [-0.16839715 -0.15171354]
 [-0.16546604 -0.15131114]
 [-0.17313397 -0.15236385]]


# 3. Détermination des spectres élémentaires

Dans l'espace de dimension réduite déterminé dans la question précédente, les données spectrales prennent la forme géométrique d'un simplexe. Les sommets de ce simplexe correspondent aux spectres élémentaires présents dans la scène. Implémenter l'algorithme N-FINDR décrit dans les slides accompagnant le mini-projet pour déterminer les spectres élémentaires.



In [9]:
class nfindr:

    """
    NFINDR algorithm implementation
    """

    def __init__(self, projected_data, R):

        """
        Constructor

        :param projected_data: Projected data points
        :param R: Number of endmembers
        :type projected_data: Numpy array 
        :type R: int
        """

        self.projected_data = projected_data
        self.N, self.K = self.projected_data.shape    
        self.R = R 


    def set_endmembers(self):

        """
        Initialize the NFINDR algorithm 
        """

        # Randomly select R endmembers from the data points
        
        rand_index = [np.random.randint(0, 9025) for k in range(self.R)]
        self.indexs = rand_index
        self.endmembers = self.projected_data[rand_index, :]
        first_point = self.endmembers[0, :]
        M = self.endmembers - first_point
        M = M[1:]

        # Compute the volume of the simplex
        self.V = np.abs(np.linalg.det(M))/factorial(self.R)

        

    def iterate(self):

        """
        N-FINDR Iteration
        """

        # Iterate over the endmembers
        for p in range(self.R):
            end = np.copy(self.endmembers)

            # Iterate over the data points
            for q in range(self.N):
                # Try replacing the selected endmember by the data point
                end[p,:] = self.projected_data[q, :]
                first_point = end[0, :]
                M = end - first_point
                M = M[1:]

                # Update the endmember if the volume is greater
                new_V = np.abs(np.linalg.det(M))/factorial(self.R)
                if new_V > self.V:
                    self.endmembers[p] = end[p]
                    self.V = new_V
                    self.indexs[p] = q



    def run(self):

        """
        Run the algorithm
        """

        # Randomly select endmembers within the data points
        self.set_endmembers()

        # Iterate until idempotence is reached
        flag = True
        while flag:
            end_cond = self.endmembers
            self.iterate()
            if np.array_equal(self.endmembers, end_cond):
                flag = False
  
        # Display result
        self.display_endmembers()
        self.original = data[self.indexs]
            

    def display_endmembers(self):

        """
        Scatter plot of the simplex data
        """
        print(self.endmembers)
        plt.scatter(self.endmembers[:,0], self.endmembers[:,1])
        plt.scatter(projected_data[:,0], projected_data[:, 1], s=1)
        plt.gca().set_xlim(-0.22, 0.33)
        plt.gca().set_ylim(-0.185, -0.08)
        plt.title("Simplex data")
        plt.show()
        



[[ 0.313665   -0.08553223]
 [-0.20324776 -0.15649811]
 [-0.14926574 -0.18108584]]


[0, 6449, 5167]


In [10]:
inst = nfindr(projected_data, R=3)
inst.run()
print(inst.indexs)

# 4. Détermination des abondances

Nous cherchons maintenant à identifier les abondances de chacun des spectres, c'est à dire à déterminer, en chaque point, quelle est la proportion relative de chacun des spectres élémentaires identifiés dans l'étape précédente. Pour ce faire, nous cherchons à résoudre, à chaque pixel de l'image, le problème d'optimisation suivant:

$$
\alpha := \arg \min \| Y[x, y] - \sum_{i = 1}^3 \alpha _{i}[x, y] m_i \|^2
$$

sous les contraintes 

$$
\forall i = 1,2,3, \quad \alpha_i \geq 0
$$

et 

$$
\sum_{i = 1}^3 \alpha_i = 1.
$$

Pour résoudre ce problème, appliquer un algorithme de descente de gradient projeté, tel que décrit dans les slides.

In [ ]:
class projected_gradient:

    """
    Projected gradient descent implementation for solving
    minimize ||Ma - y||^2 s.t 0 <= a <= 1.
    """

    def __init__(self, M, y):

        """
        Constructor
        
        :param M: endmembers matrix
        :param y: spectra
        """
        
        self.M = M.T
        ones = np.ones(shape=(1, 3))
        self.M = np.concatenate((self.M, ones))
        self.D, self.R = self.M.shape
        self.y = y
        self.y = np.append(self.y, 1.)
        self.a = np.zeros(self.R)
        self.f = norm(self.M @ self.a - self.y, ord = 2)
        self.alpha = 0.5
        self.beta = 0.1
        
        # self.yt = np.concatenate((self.y, np.array([1.])))
        # self.Mt = np.concatenate((self.M, np.ones((1, self.D))))

    def iterate(self):

        """
        Iteration of the projected gradient descent algorithm
        """

        # Compute the gradient
        grad_J = 2*np.dot(self.M.T, self.M @ self.a - self.y)
        next = True
        t = 0.0001
        a_copy = self.a
        while(next):
            # Proximal operator
            droite = self.f - self.alpha*t*np.power(np.linalg.norm(grad_J), 2)
            a_copy -= t*grad_J
            gauche = self.f
            t *= self.beta
            # Line search
            if  gauche <= droite:
                next = False
        self.a -= t*grad_J
        self.a[self.a < 0] = 0
        self.a[self.a > 1] = 1
        self.grad_J = grad_J
                

    def run(self, eps, maxiter):

        """
        Run the proximal gradient algorithm

        :param eps: Requested precision 
        :param maxiter: Maximal number of iteration
        """
        iter = 0
        while iter < maxiter:
            self.iterate()
            iter += 1
            if np.linalg.norm(self.grad_J) < eps:
                return self.a    

In [ ]:
test = projected_gradient(inst.original, data[3000, :])
rep = test.run(0.01, 100000)
print(rep)

In [ ]:
def display_abundances(abundances):

    """
    Display the abundances maps

    :param abundances: Abundances map
    :type abunbances: Numpy array
    """
    shape = abundances.shape
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            
            pix = data[i*shape[0] + j]
            proj = projected_gradient(inst.original, pix)
            ab_pix = proj.run(0.01, 10000)
            abundances[i, j] = ab_pix
            
    plt.figure("Abundances map")
    plt.title("Abundances map")
    plt.gca().pcolormesh(abundances)
    plt.show()


In [ ]:
# Abundances estimation
endmembers = inst.endmembers
N, D = projected_data.shape

R = D + 1 # Number of endmembers
abundances = np.zeros((N, R))
    
# Trick for handling the sum-to-one constraint (see slides)

display_abundances(abundances.reshape((95, 95, 3)))